### 选出活跃用户

In [1]:
from select import select
from odps import ODPS
from odps import options
from odps.df import DataFrame
import pandas as pd
import numpy as np

In [2]:
ACCESS_ID = 'LTAI5tHDArybZRnXaPS3pdkJ'
SECRET_ACCESS_KEY = 'lXXLM2U1dB3ExgKquGUwdGb88WHqkN'
ODPS_PROJECT = 'OpenDigger_prod_dev'
ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'

In [3]:
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False

In [4]:
result = o.execute_sql("select actor_id, count(*) from ods_github_log where pt>='20210101' and pt<'20220101' group by actor_id order by count(*) DESC limit 10000;",hints={'odps.sql.allow.fullscan': 'true'})
res_users = result.open_reader().to_pandas()

In [7]:
# res_users.to_csv('./res_users.csv',index=0)

### 根据活跃用户找合作的人

In [10]:
import time

In [5]:
users = res_users[["actor_id"]].values.reshape(10000).tolist()
# print(users)

In [ ]:
antusers = {}
i = 0
for user in users:
    i += 1
    antusers[user] = i
print(antusers)

In [14]:
for user in users:
    user = users[4999]
    s = str(user)
    print(s)
    print(time.time())
    result = o.execute_sql("select first.actor_id, count(*) from ods_github_log first, ods_github_log second, (select actor_id from ods_github_log where pt>='20210101' and pt<'20220101' group by actor_id order by count(*) DESC limit 10000)as temp(Aid) where first.actor_id=temp.Aid and first.pt>='20210101' and first.pt<'20220101' and second.pt>='20210101' and second.pt<'20220101' and first.repo_id=second.repo_id and second.actor_id=" + s + " group by first.actor_id order by first.actor_id;",hints={'odps.sql.allow.fullscan': 'true','odps.sql.validate.orderby.limit': 'false'})
    print(time.time())
    links = result.open_reader().to_pandas()
    print(links)
    break;

574281
1648461333.5610251
1648461385.648948
     actor_id  _c1
0        1090    3
1        3856    1
2       64669    1
3       81942   12
4      116184    3
..        ...  ...
184  81090657    1
185  83625655    1
186  86180257    2
187  86299650    4
188  92409154   10

[189 rows x 2 columns]
